In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Harris Corner Detector:

- Apply Gaussian window function to avoid noisy response
- considers small shifts by first order taylor series expansion
- Determine intensity change by analyzing the eigenvalues of second-moment matrix

- criteria for Harris response
    - R_xy = det(M) - K * trace (M)^2 > threshold
    

In [17]:
def harrisCorners(image, ksize, windowSize, k, thresholdRatio):

    # get dimension of input image (image : grayscale image)
    height, width = image.shape

    # define output matrix for harris response
    harris_response = np.zeros((height, width), np.float32)

    # define array for indicies of thresholded harris response
    corners = []

    # compute image gradient dx, and dy : floating point precision is needed => set ddepth of output to float
    dx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize)
    dy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize)

    # Compute squared gradient dxx, dyy and dxy
    dxx = dx * dx
    dyy = dy * dy
    dxy = dx * dy

    # Apply gaussian window function to squared gradients, here we GaussianBlur()
    dx2 = cv2.GaussianBlur(dxx, (3,3),0)
    dy2 = cv2.GaussianBlur(dyy, (3,3),0)
    dxy2 = cv2.GaussianBlur(dxy, (3,3),0)

    # Compute the sums of the squared gradient at each pixel p(y,x)
    offset = int(windowSize/2)
    for y in range(offset, height-offset): 
        for x in range(offset, width-offset):

            # calculate sum of squared gradients
            Sx2 = np.sum(dx2[y-offset:y+offset, x-offset:x+offset])
            Sy2 = np.sum(dy2[y-offset:y+offset, x-offset:x+offset])
            Sxy = np.sum(dxy2[y-offset:y+offset, x-offset:x+offset])

            # define the second moment matrix 
            M = np.array([[Sx2, Sxy], [Sxy, Sy2]])

            # calculate the response function 
            det = np.linalg.det(M) # det = Sx2*Sy2 - Sxy**2
            trace = np.matrix.trace(M) # trace = Sx2 + Sy2
            harris_response[y, x] = det - k*(trace**2) # Harris response function

    # show response matrix
    # cv2.imshow("Response Matrix", harris_response)

    # determine threshold using maximum corner response
    threshold = thresholdRatio * harris_response.max()

    # Apply threshold and get corners
    corners = np.argwhere(harris_response > threshold)

     # binarize response matrix by threshold for display purpose
    harris_response[harris_response <= threshold] = 0
    harris_response[harris_response > threshold] = 255
    # cv2.imshow("Response matrix", harris_response)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return corners


In [20]:
# load image
img = cv2.imread('Image/house1.jpg',1)
# plt.imshow(img)


img_copencv = img.copy()

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# define kernel
KSobel_Size = 3
window_Size = 3

# define harris free parameters k and threshold ratio
k =0.06  # 0.04 - 0.06
threshold_ratio = 0.01

# find corners by harris detector
harris_Corners = harrisCorners(img_gray, KSobel_Size, window_Size, k, threshold_ratio)

# highlight harris corner in input image
for x,y in harris_Corners:
    # img[x,y] = [0,0,255]
    cv2.circle(img, (y,x), 3, (0,0,255))

# show the images
cv2.imshow("Harris Detector", img)

cv2.waitKey(0)
cv2.destroyAllWindows() 


In [25]:
# Using Harris corner detector from OpenCV

corners = cv2.cornerHarris(img_gray, window_Size-1, KSobel_Size, k)

index = np.argwhere(corners > threshold_ratio * corners.max())

for x,y in index:
    cv2.circle(img_copencv, (y,x), 3, (0,0,255))

# binarize OpenCV Harris corners for display purpose
corners[corners <= threshold_ratio*corners.max()] = 0
corners[corners > threshold_ratio*corners.max()] = 255


cv2.imshow("OpneCV Harris corner detector", img_copencv)
cv2.waitKey(0)
cv2.destroyAllWindows() 




Benefits for Harris corner detectors:
- invariant to image rotation
- partially invariant to affine intensity changes
- not invariant to scaling

Feature Description

- Feature Detector:

    • calculates locations of significant areas in an image (e.g., corners)
    
    • no other information about the detected features

- Feature Descriptor:

    • local description of surrounding pixels

    • outputs feature vectors

    • numerical "fingerprint"

    • ideally: invariant under image transformation
    
    • descriptor vector for each feature point

Feature Matching:

• Descriptors are compared across images to identify similar features

• Useful to solve N-view correspondence problems

Process of Feature Matching consists of:
1. Find interest points (feature detection)
2. Compute descriptor vector for each feature point (feature description)
3. Compare descriptors across images to identify similar pixels (feature matching)

Invariances of Harris Corner Descriptor:

    • not invariant to rotation

    • not invariant to scaling


# SIFT - scale invariant feature transform

Characteristics of SIFT:

• detecting keypoints in scale space

• assigning an orientation to each keypoint

• descriptor to each keypoint (16x16 neighbourhood around keypoint)

In [31]:

# load image

image1 = cv2.imread('Image/house1.jpg')
im1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2 = cv2.imread('Image/house2.jpg')
im2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# construct a SIFT object
sift = cv2.SIFT_create()
orb = cv2.ORB_create()


# detect SIFT keypoints and compute descriptors from keypoints
# kp1, des1 = sift.detectAndCompute(im1, None)
# kp2, des2 = sift.detectAndCompute(im2, None)

# also we can detect kp, and compute descriptors separetly for ORB
kp1 = orb.detect(im1)
kp1, des1 = orb.compute(im1, kp1)
kp2 = orb.detect(im2)
kp2, des2 = orb.compute(im2, kp2)


# detect 

# Brute force matcher
matcher = cv2.BFMatcher()
matches = matcher.match(des1, des2)

# sort matches in the order of their distance
matches = sorted(matches, key=lambda x:x.distance)

# draw first 100 matches
cv2.drawKeypoints(image1, kp1, image1)
cv2.drawKeypoints(image2, kp2, image2)
matched_image = cv2.drawMatches(image1, kp1, image2, kp2, matches[:10], 0)

cv2.imshow("Feature matching", matched_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

SURF (Speeded-Up Robust Features):

    • fast SIFT

    • approximates Laplacian of Gaussian with Box Filters
    
    • uses wavelets responses for orientation assignment

In [35]:
# # load image
# image1 = cv2.imread('Image/house1.jpg')
# img_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)


# # create SURF object
# surf = cv2.SURF()

# # detect keypoints and compute descriptors with SURF
# kp, des = surf.detectAndCompute(img_gray, None)

# # draw keypoints on top of the input image
# cv2.drawKeypoints(img, kp, img)
# cv2.imshow("SURF", img)


# cv2.waitKey(0)
# cv2.destroyAllWindows()

FAST (Features from Accelerated Segment Test):
• for real-time application (e.g., SLAM for mobile robots)
• is a feature detector, not a descriptor extractor

In [2]:
# # # load image
# image1 = cv2.imread('Image/house1.jpg')
# img_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

# # create FAST object for feature detection
# fast = cv2.FastFeatureDetector()

# # detect keypoints with FAST
# kp = fast.detect(img_gray, None)

# # create BRISK object for feature description
# # BRISK: Binary Robust Invariant Scalable Keypoints
# br = cv2.BRISK_create()

# # compute the descriptors with BRISK
# kp, des = br.compute(img_gray, kp)

# cv2.imshow(kp)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Conclusion:

- SIFT (Scale Invariant Feature Transform)

- SURF (Speeded-Up Robust Features)

- FAST (Features from Accelerated Segment Test)

- BRIEF (Binary Robust Independent Elementary features)

- ORB (Oriented FAST and Rotated BRIEF):

    • combines FAST keypoint detector and BRIEF descriptor